In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import ee
import geemap
import numpy as np

In [2]:
# Authenticate and initialize the GEE API
ee.Authenticate()
ee.Initialize()

In [3]:
countries = ee.FeatureCollection('FAO/GAUL/2015/level0')

# Filter for a specific country, such as the United States
us_states = countries.filter(ee.Filter.eq('ADM0_NAME', 'United States'))

# Print the GEE object to confirm
print(f'GEE object type: {type(us_states)}')

GEE object type: <class 'ee.featurecollection.FeatureCollection'>


In [5]:
m = geemap.Map(center=[40, -100], zoom=4)
dem = ee.Image("USGS/SRTMGL1_003")
dem_vis = {
    "min": 0,
    "max": 4000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}
m.add_layer(dem, dem_vis, "SRTM DEM", True, 0.6)
states = ee.FeatureCollection("TIGER/2018/States").filterBounds(
    ee.Geometry.BBox(-127, 23, -66, 50)
)
style = {"fillColor": "00000000"}
m.add_layer(states.style(**style), {}, "US States")
m

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', tr…

In [9]:
dem_stats = geemap.zonal_stats(
    dem, states, stats_type="MEAN", scale=1000, return_fc=True, verbose=False
)
dem_df = geemap.ee_to_df(dem_stats)
dem_df.sort_values("mean", ascending=False).head()

,ALAND,AWATER,DIVISION,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,MTFCC,NAME,REGION,STATEFP,STATENS,STUSPS,mean
41,268422891711,1181621593,8,A,08,+38.9938482,-105.5083165,00,G4000,Colorado,4,08,01779779,CO,2095.192956
42,251458544898,1867670745,8,A,56,+42.9896591,-107.5443922,00,G4000,Wyoming,4,56,01779807,WY,1949.331152
40,212886221680,6998824394,8,A,49,+39.3349925,-111.6563326,00,G4000,Utah,4,49,01455989,UT,1836.623316
39,314196306401,728776523,8,A,35,+34.4346843,-106.1316181,00,G4000,New Mexico,4,35,00897535,NM,1765.391363
43,284329506470,2047206072,8,A,32,+39.3310928,-116.6151469,00,G4000,Nevada,4,32,01779793,NV,1690.774989


In [10]:
geemap.bar_chart(
    dem_df,
    x="NAME",
    y="mean",
    x_label="State",
    y_label="Mean Elevation (m)",
    title="Average Elevation by State",
)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'State=%{x}<br>Mean Elevation (m)=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Colorado', 'Wyoming', 'Utah', 'New Mexico', 'Nevada', 'Idaho',
                          'Arizona', 'Montana', 'Oregon', 'California', 'Nebraska',
                          'South Dakota', 'Washington', 'Kansas', 'North Dakota', 'West Virginia',
                          'Texas', 'Oklahoma', 'Pennsylvania', 'Vermont', 'Minnesota',
                          'New Hampshire', 'Iowa', 'New York', 'Wisconsin', 'Tennessee',
                          'Virginia', 'Ohio', 'Missouri', 'Kentucky', 'Michigan', 'Indiana',
                          'Maine', 'North Carolina', 'Illinois', 'Arkansas', 'Georgia', 'Alabama',
                          'Connecticut', 'Massachusetts', 'Maryland', 'South Carolina',
                          'Mississippi', 'New Jersey', 'Rhode Island', 'District of Columbia',
                          'Louisiana', 'Florida', 'Delaware'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('9Rcby2JeoEDcX5QZU3WeQPAwbEZ+sp' ... 'Z4x9w+QDhPV93gNzVA7kbErfXOK0A='),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'group',
               'height': 500,
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Average Elevation by State'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'State'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Mean Elevation (m)'}}}
})

In [11]:

dem_gdf = geemap.ee_to_gdf(dem_stats)
dem_gdf.head()

,geometry,ALAND,AWATER,DIVISION,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,MTFCC,NAME,REGION,STATEFP,STATENS,STUSPS,mean
0,"MULTIPOLYGON (((-71.90725 41.30447, -71.87987 ...",2677779902,1323670487,1,A,44,+41.5974187,-071.5272723,00,G4000,Rhode Island,1,44,01219835,RI,57.769871
1,"POLYGON ((-72.55711 42.85322, -72.55707 42.852...",23189413166,1026675248,1,A,33,+43.6726907,-071.5843145,00,G4000,New Hampshire,1,33,01779794,NH,347.549097
2,"POLYGON ((-73.4379 44.04514, -73.43759 44.0438...",23874175944,1030416650,1,A,50,+44.0685773,-072.6691839,00,G4000,Vermont,1,50,01779802,VT,373.218511
3,"POLYGON ((-73.72776 41.1007, -73.72686 41.0994...",12542497068,1815617571,1,A,09,+41.5798637,-072.7466572,00,G4000,Connecticut,1,09,01779780,CT,133.111310
4,"POLYGON ((-71.08391 45.30533, -71.08391 45.305...",79887426037,11746549764,1,A,23,+45.4092843,-068.6666160,00,G4000,Maine,1,23,01779787,ME,217.098618


NameError: name 'in_colab_shell' is not defined